**Install Required Libraries**

In [6]:
!pip install pymc arviz pandas numpy matplotlib seaborn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

import pandas as pd
import numpy as np

df = pd.read_csv("datasets/resource_dataset_with_id_category_storm_features.csv")  # Replace with your actual file

df['maximum_sustained_wind_knots_max_norm'] = (df['maximum_sustained_wind_knots_max'] - df['maximum_sustained_wind_knots_max'].mean()) / df['maximum_sustained_wind_knots_max'].std()
df['maximum_sustained_wind_knots_mean_norm'] = (df['maximum_sustained_wind_knots_mean'] - df['maximum_sustained_wind_knots_mean'].mean()) / df['maximum_sustained_wind_knots_mean'].std()
df['central_pressure_mb_min_norm'] = (df['central_pressure_mb_min'] - df['central_pressure_mb_min'].mean()) / df['central_pressure_mb_min'].std()
df['central_pressure_mb_mean_norm'] = (df['central_pressure_mb_mean'] - df['central_pressure_mb_mean'].mean()) / df['central_pressure_mb_mean'].std()
df['radius_of_max_wind_nm_max_norm'] = (df['radius_of_max_wind_nm_max'] - df['radius_of_max_wind_nm_max'].mean()) / df['radius_of_max_wind_nm_max'].std()
df['radius_of_max_wind_nm_mean_norm'] = (df['radius_of_max_wind_nm_mean'] - df['radius_of_max_wind_nm_mean'].mean()) / df['radius_of_max_wind_nm_mean'].std()

y = df["shelters"].values
X = df[['maximum_sustained_wind_knots_max_norm', 'maximum_sustained_wind_knots_mean_norm', 'central_pressure_mb_min_norm', 'central_pressure_mb_mean_norm', 'radius_of_max_wind_nm_max_norm', 'radius_of_max_wind_nm_mean_norm']]


In [7]:
df.head()

,storm_name,category,year,shelters,meals,water_gallons,fuel_gallons,storm_id,maximum_sustained_wind_knots_max,maximum_sustained_wind_knots_mean,central_pressure_mb_min,central_pressure_mb_mean,radius_of_max_wind_nm_max,radius_of_max_wind_nm_mean,maximum_sustained_wind_knots_max_norm,maximum_sustained_wind_knots_mean_norm,central_pressure_mb_min_norm,central_pressure_mb_mean_norm,radius_of_max_wind_nm_max_norm,radius_of_max_wind_nm_mean_norm
0,CHARLEY,3,2004,250,2000000,300000,100000,AL032004,130,109.444444,941.0,957.888889,5.0,5.0,0.703856,0.854193,-0.043873,-0.199724,-1.248173,-1.207079
1,FRANCES,3,2004,250,3500000,1500000,200000,AL062004,95,70.000000,958.0,971.600000,30.0,30.0,-0.938474,-0.776530,0.837567,0.549806,0.780108,1.001510
2,IVAN,3,2004,120,2000000,700000,200000,AL092004,110,47.272727,931.0,990.545455,25.0,25.0,-0.234619,-1.716128,-0.562366,1.585477,0.374452,0.559792
3,JEANNE,-1,2004,200,4000000,1500000,200000,AL112004,105,73.333333,950.0,965.111111,45.0,45.0,-0.469237,-0.638723,0.422772,0.195085,1.997077,2.326664
4,DENNIS,-1,2005,70,1000000,500000,100000,AL042005,125,111.250000,930.0,941.125000,5.0,5.0,0.469237,0.928838,-0.614216,-1.116138,-1.248173,-1.207079


In [23]:
import pymc as pm
pm.set_backend("numpy")

with pm.Model() as model:
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    beta_maximum_sustained_wind_knots_max = pm.Normal('beta_maximum_sustained_wind_knots_max', mu=0, sigma=2)
    beta_maximum_sustained_wind_knots_mean = pm.Normal('beta_maximum_sustained_wind_knots_mean', mu=0, sigma=2)
    beta_central_pressure_mb_min = pm.Normal('beta_central_pressure_mb_min', mu=0, sigma=2)
    beta_central_pressure_mb_mean = pm.Normal('beta_central_pressure_mb_mean', mu=0, sigma=2)
    beta_radius_of_max_wind_nm_max = pm.Normal('beta_radius_of_max_wind_nm_max', mu=0, sigma=2)
    beta_radius_of_max_wind_nm_mean = pm.Normal('beta_radius_of_max_wind_nm_mean', mu=0, sigma=2)

    # Prior for standard deviation (must use sigma if using mu)
    sigma = pm.HalfNormal("sigma", sigma=10)

    # Linear model for log(mu) to ensure positivity
    mu = pm.math.exp(
        intercept +
        beta_maximum_sustained_wind_knots_max * X['maximum_sustained_wind_knots_max_norm'].values +
        beta_maximum_sustained_wind_knots_mean * X['maximum_sustained_wind_knots_mean_norm'].values +
        beta_central_pressure_mb_min * X['central_pressure_mb_min_norm'].values +
        beta_central_pressure_mb_mean * X['central_pressure_mb_mean_norm'].values +
        beta_radius_of_max_wind_nm_max * X['radius_of_max_wind_nm_max_norm'].values +
        beta_radius_of_max_wind_nm_mean * X['radius_of_max_wind_nm_mean_norm'].values
    )

    # Gamma likelihood
    y_obs = pm.Gamma("y_obs", mu=mu, sigma=sigma, observed=y)

    # Sample from posterior
    trace = pm.sample(3000, tune=1000, target_accept=0.95, return_inferencedata=True)


AttributeError: module 'pymc' has no attribute 'set_backend'

**Define Bayesian Gamma Regression Model**

In [ ]:

import pymc as pm

with pm.Model() as model:
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    beta_wind = pm.Normal("beta_wind", mu=0, sigma=2)
    beta_pressure = pm.Normal("beta_pressure", mu=0, sigma=2)
    beta_radius = pm.Normal("beta_radius", mu=0, sigma=2)
    alpha = pm.HalfNormal("alpha", sigma=10)

    mu = pm.math.exp(
        intercept +
        beta_wind * X["wind_speed_norm"].values +
        beta_pressure * X["pressure_norm"].values +
        beta_radius * X["radius_norm"].values
    )

    y_obs = pm.Gamma("y_obs", mu=mu, alpha=alpha, observed=y)

    trace = pm.sample(3000, tune=1000, target_accept=0.95, return_inferencedata=True)


**Summarize Posterior Estimates**

In [ ]:

df_pred = pd.read_csv("800_hurricanes.csv")  # Replace with actual file path

df_pred['maximum_sustained_wind_knots_max_norm'] = (df_pred['maximum_sustained_wind_knots_max'] - df_pred['maximum_sustained_wind_knots_max'].mean()) / df_pred['maximum_sustained_wind_knots_max'].std()
df_pred['maximum_sustained_wind_knots_mean_norm'] = (df_pred['maximum_sustained_wind_knots_mean'] - df_pred['maximum_sustained_wind_knots_mean'].mean()) / df_pred['maximum_sustained_wind_knots_mean'].std()
df_pred['central_pressure_mb_min_norm'] = (df_pred['central_pressure_mb_min'] - df_pred['central_pressure_mb_min'].mean()) / df_pred['central_pressure_mb_min'].std()
df_pred['central_pressure_mb_mean_norm'] = (df_pred['central_pressure_mb_mean'] - df_pred['central_pressure_mb_mean'].mean()) / df_pred['central_pressure_mb_mean'].std()
df_pred['radius_of_max_wind_nm_max_norm'] = (df_pred['radius_of_max_wind_nm_max'] - df_pred['radius_of_max_wind_nm_max'].mean()) / df_pred['radius_of_max_wind_nm_max'].std()
df_pred['radius_of_max_wind_nm_mean_norm'] = (df_pred['radius_of_max_wind_nm_mean'] - df_pred['radius_of_max_wind_nm_mean'].mean()) / df_pred['radius_of_max_wind_nm_mean'].std()

with model:
    pm.set_data({
        'maximum_sustained_wind_knots_max_norm': df_pred['maximum_sustained_wind_knots_max_norm'].values,
        'maximum_sustained_wind_knots_mean_norm': df_pred['maximum_sustained_wind_knots_mean_norm'].values,
        'central_pressure_mb_min_norm': df_pred['central_pressure_mb_min_norm'].values,
        'central_pressure_mb_mean_norm': df_pred['central_pressure_mb_mean_norm'].values,
        'radius_of_max_wind_nm_max_norm': df_pred['radius_of_max_wind_nm_max_norm'].values,
        'radius_of_max_wind_nm_mean_norm': df_pred['radius_of_max_wind_nm_mean_norm'].values
    })

    posterior_predictive = pm.sample_posterior_predictive(trace, var_names=["y_obs"])


**Trace Plots**

In [ ]:

import matplotlib.pyplot as plt

az.plot_trace(trace, figsize=(12, 8))
plt.tight_layout()
plt.show()


**Posterior Predictive Sampling for New Hurricanes**

In [ ]:

df_pred = pd.read_csv("800_hurricanes.csv")  # Replace with actual file path

# Normalize using the same statistics from the training set
for col in features:
    df_pred[f"{col}_norm"] = (df_pred[col] - df[col].mean()) / df[col].std()

with model:
    pm.set_data({
        "wind_speed_norm": df_pred["wind_speed_norm"].values,
        "pressure_norm": df_pred["pressure_norm"].values,
        "radius_norm": df_pred["radius_norm"].values
    })

    posterior_predictive = pm.sample_posterior_predictive(trace, var_names=["y_obs"])


**Analyze and Save Predictive Output**

In [ ]:

pred_means = posterior_predictive["y_obs"].mean(axis=0)
hdi = az.hdi(posterior_predictive["y_obs"], hdi_prob=0.94)

df_pred["shelters_pred_mean"] = pred_means
df_pred["shelters_95_low"] = hdi[:, 0]
df_pred["shelters_95_high"] = hdi[:, 1]

df_pred.to_csv("synthetic_shelters_predictions.csv", index=False)
